In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import optimize
from tqdm import tqdm

In [5]:
from BHDVCStf import BHDVCS
import utilities as uts

In [6]:
bhdvcs = BHDVCS()

In [7]:
data = pd.read_csv('dvcs_psuedo.csv')

In [8]:
def produceLHS(data, bhdvcs):
    bhdvcs.SetKinematics(data['QQ'], data['x_b'], data['t'], data['k'])
    bh = bhdvcs.GetBHUUxs(data['phi_x'], data['F1'], data['F2'])
    interference = data['F'] - data['dvcs'] - bh
    interference /= bhdvcs.Gamma
    interference *= data['QQ']
    interference *= np.abs(data['t'])
    a, b = bhdvcs.ABcon(data['phi_x'], data['F1'], data['F2'])
    interference /= b
    return interference

In [9]:
def produceLHSerr(data, bhdvcs):
    bhdvcs.SetKinematics(data['QQ'], data['x_b'], data['t'], data['k'])
    err = data['errF']
    err /= bhdvcs.Gamma
    err *= data['QQ']
    err *= np.abs(data['t'])
    a, b = bhdvcs.ABcon(data['phi_x'], data['F1'], data['F2'])
    err /= b
    return np.abs(err)

In [10]:
def aoverb(data, bhdvcs):
    bhdvcs.SetKinematics(data['QQ'], data['x_b'], data['t'], data['k'])
    a, b = bhdvcs.ABcon(data['phi_x'], data['F1'], data['F2'])
    return a/b

In [37]:
def linear_curve_fit(data, ReH, ReE):
    bhdvcs.SetKinematics(data[:, 0], data[:, 1], data[:, 2], data[:, 3])
    a, b = bhdvcs.ABcon(data[:, 4], data[:, 5], data[:, 6])
    #tau = -data['t']/(4*bhdvcs.M2)
    gm = data[:, 5] + data[:, 6]
    return (a/b)*(data[:, 5]*ReH + bhdvcs.tau*data[:, 6]*ReE) + gm*(ReH + ReE)

In [33]:
def combined_function_chisq(cffs, data):
    ReH, ReE, ReHtilde = cffs
    dftu = data.copy()
    dftu['F'] = bhdvcs.TotalUUXS(np.array(data[['phi_x', 'k', 'QQ', 'x_b', 't', 'F1', 'F2', 'dvcs']]), ReH, ReE, ReHtilde)
    yhat1 = produceLHS(dftu, bhdvcs)
    X = np.array(data[['QQ', 'x_b', 't', 'k', 'phi_x', 'F1', 'F2']])
    yhat2 = linear_curve_fit(X, ReH, ReE)
    chisq1 = np.sum(((yhat1 - data['F'])/data['errF'])**2)
    chisq2 = np.sum(((yhat2 - data['F'])/data['errF'])**2)
    return (chisq1 + chisq2)/2

In [ ]:
def produceLHS(data, bhdvcs):
    bhdvcs.SetKinematics(data['QQ'], data['x_b'], data['t'], data['k'])
    bh = bhdvcs.GetBHUUxs(data['phi_x'], data['F1'], data['F2'])
    interference = data['F'] - data['dvcs'] - bh
    interference /= bhdvcs.Gamma
    interference *= data['QQ']
    interference *= np.abs(data['t'])
    a, b = bhdvcs.ABcon(data['phi_x'], data['F1'], data['F2'])
    interference /= b
    return interference

def produceLHSerr(data, bhdvcs):
    bhdvcs.SetKinematics(data['QQ'], data['x_b'], data['t'], data['k'])
    err = data['errF']
    err /= bhdvcs.Gamma
    err *= data['QQ']
    err *= np.abs(data['t'])
    a, b = bhdvcs.ABcon(data['phi_x'], data['F1'], data['F2'])
    err /= b
    return np.abs(err)

def aoverb(data, bhdvcs):
    bhdvcs.SetKinematics(data['QQ'], data['x_b'], data['t'], data['k'])
    a, b = bhdvcs.ABcon(data['phi_x'], data['F1'], data['F2'])
    return a/b

def linear_curve_fit(data, ReH, ReE):
    bhdvcs.SetKinematics(data[:, 0], data[:, 1], data[:, 2], data[:, 3])
    a, b = bhdvcs.ABcon(data[:, 4], data[:, 5], data[:, 6])
    #tau = -data['t']/(4*bhdvcs.M2)
    gm = data[:, 5] + data[:, 6]
    return (a/b)*(data[:, 5]*ReH + bhdvcs.tau*data[:, 6]*ReE) + gm*(ReH + ReE)

def combined_function_chisq(cffs, data):
    ReH, ReE, ReHtilde = cffs
    dftu = data.copy()
    dftu['F'] = bhdvcs.TotalUUXS(np.array(data[['phi_x', 'k', 'QQ', 'x_b', 't', 'F1', 'F2', 'dvcs']]), ReH, ReE, ReHtilde)
    yhat1 = produceLHS(dftu, bhdvcs)
    X = np.array(data[['QQ', 'x_b', 't', 'k', 'phi_x', 'F1', 'F2']])
    yhat2 = linear_curve_fit(X, ReH, ReE)
    chisq1 = np.sum(((yhat1 - data['F'])/data['errF'])**2)
    chisq2 = np.sum(((yhat2 - data['F'])/data['errF'])**2)
    return (chisq1 + chisq2)/2